## Title - Master's Thesis
#### Author : Matthieu Grenier

This notebook provides the code associated with the Master's thesis "". The code was written in Julia 1.11.6.

***

#### The consumption-saving problem

As per the article of Maliar et al.(2021), we overlook the consumption-saving problem and solve using various methods. A baseline result is obtained using non-linear solvers, and we proceed by comparing 4 options :
- Perturbation methods
- Projection methods
- Deep Learning methods à la Maliar et al.(2021)
- Enhanced DL method


This example is fairly simple, but allows for proper representation of how neural networks can be useful when dealing with kinks in the decision rule. Such kinks arise in most dynamic economic models when a given constraint is binding (for instance the implementation of the ZLB in the HANK model, see ???).

This notebook was written in Julia. It is an open-source language, designed for high performance. It supports parallelization and multiple dispatch, which happens to be ideal for the sake of the thesis. The code outputs, graphs and tables are reproducible by a third-party. To do so, the reader is welcome to make use of the Julia environment associated with the notebook. Additional informations can be found in the [Pkg.jl documentation](https://pkgdocs.julialang.org/v1/environments/)

In [1]:
using Pkg
Pkg.activate(mktempdir());

  Activating new project at `C:\Users\grnma\AppData\Local\Temp\jl_8078lL`


In [2]:
# Required packages for this notebook

using LinearAlgebra, Statistics, Random, Distributions, Printf, BenchmarkTools
using QuantEcon, Interpolations, FastChebInterp, NLsolve, Optim, QuadGK
using Plots, LaTeXStrings
using ForwardDiff

ArgumentError: ArgumentError: Package BenchmarkTools not found in current path.
- Run `import Pkg; Pkg.add("BenchmarkTools")` to install the BenchmarkTools package.

***

### A first model to grasp the efficiency of the DL method

An agent aims to maximise its consumption under a borrowing constraint. Its program is :

$$
\begin{equation*}
\max_{(c_t), (w_t)}  \sum_{t=0}^\infty \beta^t u(c_t) \quad
\\[5ex]
\text{s. t. } 
\begin{cases}
w_{t+1} = r(w_t - c_t) + e^{y_t} \\
c_t \leq w_t
\end{cases}
\end{equation*}
$$

$y_t$ follows an AR(1) process : $y_{t+1} = \rho y_t + \sigma \epsilon_t, \; \epsilon_t \sim \mathcal{N}(0, 1)$

This program can be extended to all any type of agent : households, firms, governments.

In [ ]:
model = (;
    β = 0.9, # Discount factor
    r = 1.04, # Interest rate
    γ = 2.0, # Coefficient of relative risk aversion
    ρ = 0.0, # Autoregressive parameter for income shocks
    σ = 0.1, # Standard deviation of the shock
    ā   = 0.0, # Borrowing limit
    Na = 500, # Number of grid points for assets
    a_min = 0.0, # Minimum asset level
    a_max = 50.0, # Maximum asset level
    )

In [ ]:
# Variants of the utility function

u(c; model = model) = (c^(1 - model.γ) - 1) / (1 - model.γ)
u_prime(c; model = model) = c^(-model.γ)
u_prime_inv(x; model = model) = x^(-1/model.γ);

In [ ]:
# Helper functions for the consumption-saving problem
# Eases the understanding of each algorithm detailed below

clamp_to_budget(c, a, y) = min(c, y + model.r*a - model.ā)  # enforce a' ≥ ā  (⇒ c ≤ w)
asset_grid = range(model.a_min, model.a_max; length=model.Na) |> collect;

***

First, let's compute a high-accuracy solution to be our baseline throughout the notebook. We rely on the Endongenous Grid Method (EGM) introduced by Carroll in 2005 (insert link). The main idea of EDG is to generate an exogeneous grid of assets, which yields under invertability of the utility function, associated levels of consumption. We can then approximate the operator K by observing the set of all pairs $(c_i, y_i)$.

In [ ]:
# -------------------------------------------------------------------
# EGM Method to solve the consumption-saving problem
# -------------------------------------------------------------------
function egm_solve(; model = model, tol=1e-8, maxit=5_000, damp=0.95)
    # Unpack model parameters
    β, R, γ = model.β, model.r, model.γ
    ρ, σ = model.ρ, model.σ
    Na, Nz = length(asset_grid), length(ygrid)
    
    # Initial guess: consume permanent income share
    c = [0.96*(y + (R-1)*a) for a in asset_grid, y in ygrid]
    a_next = similar(c)
    it = 0
    diff = 1.0
    Emu = zeros(Na, Nz)
    while it < maxit && diff > tol
        it += 1
        # expected marginal utility next period for each z today
        for iz in 1:Nz
            for jz in 1:Nz
                @inbounds Emu[:,iz] .+= Pz[iz,jz] * u_prime.(c[:,jz])
            end
        end
        # Euler: up(c_t) = β R E[up(c_{t+1})]
        c_star = u_prime_inv.(β*R .* Emu)
        # endogenous grid for a_t (m today)
        for iz in 1:Nz
            y = ygrid[iz]
            m_star = c_star[:,iz] .+ asset_grid        # m = c + a' (since a' grid is asset_grid)
            a_of_m = (m_star .- y) ./ R                # implied a today
            # Interpolate c(a,z) on regular asset_grid
            itp = interpolate((a_of_m,), c_star[:,iz], Gridded(Linear()))
            c_new = [clamp_to_budget(itp(min(max(a, first(a_of_m)), last(a_of_m))), a, y) for a in asset_grid]
            c[:,iz] = damp .* c_new .+ (1-damp) .* c[:,iz]
            a_next[:,iz] = (y .+ R .* asset_grid) .- c[:,iz]
        end
        diff = maximum(abs.(Emu)) # cheap proxy for progress
        fill!(Emu, 0.0)
    end
    return c, a_next
end

c_egm, a_next_egm = egm_solve()

#### Lifetime-reward maximisation

Maliar et al.(2021) show that tackling the lifetime-reward maximisation is equivalent to minimizing the following objective function : 

<br><br>

**Definition 2.4** (All-in-one expectation operator for lifetime reward). *Fix time horizon* $T > 0$, *parametrize a decision rule* $\varphi(\cdot; \theta)$ *and define the distribution of the random variable* $\omega \equiv (m_0, s_0, \epsilon_1, \ldots, \epsilon_T)$. *For given* $\theta$, *lifetime reward associated with the rule* $\varphi(\cdot; \theta)$ *is given by*

$$
\Xi(\theta) = \mathbb{E}_\omega[\xi(\omega; \theta)] \equiv \mathbb{E}_{(m_0, s_0, \epsilon_1, \ldots, \epsilon_T)} \left[ \left. \sum_{t=0}^T \beta^t r(m_t, s_t, \varphi(m_t, s_t; \theta)) \right] \right.,
$$

*where transitions are determined by equations (1), (2) and (3), and* $\xi$ *is an integrand.*

<br><br>


As per Maliar et al.(2021), the estimation procedure is proceeded using deep learning associated with $n \times n'$ Monte-Carlo draws.